<a href="https://colab.research.google.com/github/Ahmed-Elbagoury/Problem-Solving-Practice/blob/master/embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is based on https://github.com/rasbt/LLMs-from-scratch/tree/main

In [1]:
from torch.utils.data import Dataset, DataLoader
import torch

In [2]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.8 MB/s eta 0:00:00


In [3]:
import tiktoken

In [4]:
class MyDataSet(Dataset):

    def __init__(self, txt, tokenizer, max_len, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        for i in range (0, len(token_ids)-max_len, stride):
            input_tokens = token_ids[i: i + max_len]
            output_tokens = token_ids[i+ 1: i+max_len + 1]
            self.input_ids.append(torch.tensor(input_tokens))
            self.target_ids.append(torch.tensor(output_tokens))

    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return {"input": self.input_ids[idx], "target": self.target_ids[idx]}

In [5]:
tokenizer = tiktoken.get_encoding("gpt2")
txt = """This is a test text. This is mostly for testing the tokenization only.
Later text will test training and inferecne
"""
dataset = MyDataSet(txt, tokenizer, 3, 2)

In [6]:
for ip, op in zip(dataset.input_ids, dataset.target_ids):
    print(ip.size(), op.size())
    print(f"'{tokenizer.decode([elem.item() for elem in ip])}'",
          "-->",
          f"'{tokenizer.decode([elem.item() for elem in op])}'")
    print("-------")

torch.Size([3]) torch.Size([3])
'This is a' --> ' is a test'
-------
torch.Size([3]) torch.Size([3])
' a test text' --> ' test text.'
-------
torch.Size([3]) torch.Size([3])
' text. This' --> '. This is'
-------
torch.Size([3]) torch.Size([3])
' This is mostly' --> ' is mostly for'
-------
torch.Size([3]) torch.Size([3])
' mostly for testing' --> ' for testing the'
-------
torch.Size([3]) torch.Size([3])
' testing the token' --> ' the tokenization'
-------
torch.Size([3]) torch.Size([3])
' tokenization only' --> 'ization only.'
-------
torch.Size([3]) torch.Size([3])
' only.
' --> '.
Later'
-------
torch.Size([3]) torch.Size([3])
'
Later text' --> 'Later text will'
-------
torch.Size([3]) torch.Size([3])
' text will test' --> ' will test training'
-------
torch.Size([3]) torch.Size([3])
' test training and' --> ' training and inf'
-------
torch.Size([3]) torch.Size([3])
' and infere' --> ' inferec'
-------
torch.Size([3]) torch.Size([3])
'erecne' --> 'cne
'
-------


In [7]:
def create_dataloader(txt, max_len=256, stride=128, batch_size=4, shuffle=True,
                      drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = MyDataSet(txt, tokenizer, max_len, stride)

    return DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle,
                            drop_last= drop_last,
                            num_workers= num_workers)

In [8]:
import os
import urllib.request

file_path = "the-verdict.txt"
if not os.path.exists(file_path):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    urllib.request.urlretrieve(url, file_path)

In [9]:
with open(file_path, "r") as f:
    txt = f.read()

In [10]:
loader = create_dataloader(txt=txt, max_len=4, stride=1, batch_size=1,
                           shuffle=False)

In [11]:
data_iter = iter(loader)
first_batch = next(data_iter)
print(first_batch["input"])
print(first_batch["target"])

tensor([[  40,  367, 2885, 1464]])
tensor([[ 367, 2885, 1464, 1807]])


In [12]:
second_batch = next(data_iter)
print(second_batch["input"])
print(second_batch["target"])

tensor([[ 367, 2885, 1464, 1807]])
tensor([[2885, 1464, 1807, 3619]])


In [13]:
loader = create_dataloader(txt=txt, max_len=8, stride=4, batch_size=8,
                           shuffle=False)

In [14]:
data_iter = iter(loader)
first_data = next(data_iter)
print("Inputs:\n", first_data["input"])
print("\nTargets:\n", first_data["target"])

Inputs:
 tensor([[   40,   367,  2885,  1464,  1807,  3619,   402,   271],
        [ 1807,  3619,   402,   271, 10899,  2138,   257,  7026],
        [10899,  2138,   257,  7026, 15632,   438,  2016,   257],
        [15632,   438,  2016,   257,   922,  5891,  1576,   438],
        [  922,  5891,  1576,   438,   568,   340,   373,   645],
        [  568,   340,   373,   645,  1049,  5975,   284,   502],
        [ 1049,  5975,   284,   502,   284,  3285,   326,    11],
        [  284,  3285,   326,    11,   287,   262,  6001,   286]])

Targets:
 tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899],
        [ 3619,   402,   271, 10899,  2138,   257,  7026, 15632],
        [ 2138,   257,  7026, 15632,   438,  2016,   257,   922],
        [  438,  2016,   257,   922,  5891,  1576,   438,   568],
        [ 5891,  1576,   438,   568,   340,   373,   645,  1049],
        [  340,   373,   645,  1049,  5975,   284,   502,   284],
        [ 5975,   284,   502,   284,  3285,   326,    1

# Embedding

In [15]:
vocab_size = 6
embedding_dim = 3

In [16]:
torch.manual_seed(1234)

In [17]:
embedding_layer = torch.nn.Embedding(vocab_size, embedding_dim)

In [18]:
print(embedding_layer.weight)

Parameter containing:
tensor([[-0.1117, -0.4966, -0.7049],
        [-1.6024,  0.2891,  0.4899],
        [-0.3853, -0.7120,  0.6369],
        [-0.7141,  0.2207,  0.2463],
        [-1.3248,  0.6970, -0.6631],
        [ 1.2158, -2.5273,  1.4778]], requires_grad=True)


In [20]:
embedding_layer(torch.tensor([3]))

tensor([[-0.7141,  0.2207,  0.2463]], grad_fn=<EmbeddingBackward0>)

In [22]:
input_ids = torch.tensor([2, 3, 5, 1])
print(embedding_layer(input_ids))

tensor([[-0.3853, -0.7120,  0.6369],
        [-0.7141,  0.2207,  0.2463],
        [ 1.2158, -2.5273,  1.4778],
        [-1.6024,  0.2891,  0.4899]], grad_fn=<EmbeddingBackward0>)


# Positional Embeddins

In [23]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [34]:
max_length = 4
dataloader = create_dataloader(
    txt=txt, batch_size=8, max_len=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
element = next(data_iter)

In [35]:
print(element.keys())

dict_keys(['input', 'target'])


In [37]:
print("inputs", element["input"])
print("output", element["target"])

inputs tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
output tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [39]:
input_embeddings = token_embedding_layer(element["input"])
output_embeddings = token_embedding_layer(element["target"])

In [40]:
print("input_embeddings.shape", input_embeddings.shape)
print("output_embeddings.shape", output_embeddings.shape)

input_embeddings.shape torch.Size([8, 4, 256])
output_embeddings.shape torch.Size([8, 4, 256])


In [42]:
seq_len = max_length
pos_embedding_layer = torch.nn.Embedding(seq_len, output_dim)

print(pos_embedding_layer.weight.shape)

torch.Size([4, 256])


In [45]:
pos_embedding = pos_embedding_layer(torch.arange(max_length))
print(pos_embedding.shape)

torch.Size([4, 256])


In [48]:
final_embedding = input_embeddings + pos_embedding
print(final_embedding.shape)

torch.Size([8, 4, 256])
